### Student Information
Name: 賴迎曦 Dina Yingxi Lai

Student ID: b09303065

GitHub ID: dina496

Kaggle name: dinayingxilai

Kaggle private scoreboard snapshot:

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home exercises** in the [DM2024-Lab2-master Repo](https://github.com/didiersalazar/DM2024-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook.


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework) regarding Emotion Recognition on Twitter by this link: https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework. The scoring will be given according to your place in the Private Leaderboard ranking:
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (0.6N + 1 - x) / (0.6N) * 10 + 20 points, where N is the total number of participants, and x is your rank. (ie. If there are 100 participants and you rank 3rd your score will be (0.6 * 100 + 1 - 3) / (0.6 * 100) * 10 + 20 = 29.67% out of 30%.)   
    Submit your last submission **BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)**. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developing the model for the competition (You can use code and comment on it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained.


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)__.

## Second

I tried to train my model although it took too long (more than 20 hours) to run. Hence I tried to narrow it down by using smaller pretrained model, mixed precision training, reduced batch size, and gradient accumulation, but it still took me 8 hours to run. It resulted in a smaller sample size of 32179 rows, but then it didn't allow me to upload on Kaggle.

In [34]:
# Begin Assignment Here
import pandas as pd
import json

# Load CSV files
data_identification = pd.read_csv('data_identification.csv')
emotion = pd.read_csv('emotion.csv')
sample_submission = pd.read_csv('sampleSubmission.csv')

# Load JSON file
tweets_data = []
with open('tweets_DM.json', 'r') as file:
  for line in file:
    try:
      tweets_data.append(json.loads(line))
    except json.JSONDecodeError as e:
      print(f"Error decoding JSON: {e}")

# Create a DataFrame for tweets
tweets_df = pd.DataFrame([{
    "tweet_id": tweet["_source"]["tweet"]["tweet_id"],
    "text": tweet["_source"]["tweet"]["text"]
} for tweet in tweets_data])

In [35]:
# Merge datasets
# Match the tweet_id in the JSON with the identification and emotion files
merged_df = tweets_df.merge(data_identification, on="tweet_id", how="left")
merged_df = merged_df.merge(emotion, on="tweet_id", how="left")

`merged_df` contains `tweet_id`, `text`, `hashtags`, `identification` (train/test split), and `emotion`.

### Data Cleaning and Preprocessing
* Remove unwanted text elements
* Tokenization
* Stopword Removal
* Stemming

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download required NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    # Remove mentions, hashtags, links, and special characters
    text = re.sub(r"(@[A-Za-z0-9_]+)|(#\w+)|(\w+:\/\/\S+)|[^a-zA-Z\s]", "", text)
    text = text.lower().strip()

    # Tokenize, remove stopwords, and lemmatize
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words]

    return " ".join(words)

# Apply preprocessing to the text data
train_data['clean_text'] = train_data['text'].apply(preprocess_text)
test_data['clean_text'] = test_data['text'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
<ipython-input-4-1e45ab3cb99f>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['clean_text'] = train_data['text'].apply(preprocess_text)
<ipython-input-4-1e45ab3cb99f>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['clean_text'] = test_data['text'].apply(preprocess_text)


In [36]:
import re

def clean_tweet(tweet):
    tweet = re.sub(r"http\S+|www\S+|https\S+", "", tweet, flags=re.MULTILINE)  # Remove URLs
    tweet = re.sub(r"@\w+|\#\w+", "", tweet)  # Remove mentions and hashtags
    tweet = re.sub(r"<LH>", "", tweet)  # Remove specific tokens
    tweet = re.sub(r"[^\w\s]", "", tweet)  # Remove punctuation
    tweet = tweet.lower()  # Lowercase
    return tweet

merged_df["cleaned_text"] = merged_df["text"].apply(clean_tweet)

### Feature engineering
#### BoW

In [37]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1000)
X_bow = vectorizer.fit_transform(merged_df['cleaned_text'])

#### TF-IDF

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_tfidf = tfidf_vectorizer.fit_transform(merged_df['cleaned_text'])

#### Word2Vec

In [39]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

merged_df['tokens'] = merged_df['cleaned_text'].apply(word_tokenize)

# Assuming merged_df has a 'cleaned_text' column:
merged_df['tokens'] = merged_df['cleaned_text'].apply(word_tokenize)

from gensim.models import Word2Vec
model = Word2Vec(sentences=merged_df['tokens'], vector_size=100, window=5, min_count=1, workers=4)
word_vectors = model.wv

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


### Modelling
#### Logistic Regression

In [40]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [41]:
from sklearn.model_selection import train_test_split

# Separate train and test data
train_data = merged_df[merged_df["identification"] == "train"]
test_data = merged_df[merged_df["identification"] == "test"]

# Extract features and labels
X_train = train_data["cleaned_text"]
y_train = train_data["emotion"]
X_test = test_data["cleaned_text"]
y_test = test_data["emotion"]

I notice that y_test does not contain any emotions, indicating that the test data might be unlabeled. For validation purposes, I need to create a validation set from the training data.

In [47]:
from sklearn.model_selection import train_test_split

# Split the labeled training data
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

**Bag-of-Words (BoW)**

In [44]:
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)
model.fit(X_train_bow, y_train)
y_pred_bow = model.predict(X_test_bow)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


**TF-IDF**

In [45]:
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
model.fit(X_train_tfidf, y_train)
y_pred_tfidf = model.predict(X_test_tfidf)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


**Word Embeddings**

In [48]:
import numpy as np

def get_avg_word2vec(tokens_list, model, vector_size):
    # Access word vectors directly using model[word] instead of model.wv[word]
    return np.array([np.mean([model[word] for word in tokens if word in model] or [np.zeros(vector_size)], axis=0)
                     for tokens in tokens_list])

X_train_tokens = merged_df.loc[X_train.index, 'tokens']
X_test_tokens = merged_df.loc[X_test.index, 'tokens']
X_train_w2v = get_avg_word2vec(X_train_tokens, word_vectors, vector_size=100)
X_test_w2v = get_avg_word2vec(X_test_tokens, word_vectors, vector_size=100)
model.fit(X_train_w2v, y_train)
y_pred_w2v = model.predict(X_test_w2v)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


**Compare performance**

In [57]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Initialize the logistic regression model
model = LogisticRegression(max_iter=1000)

# Store results for comparison
results = {}

# Function to train and evaluate a model for a given feature representation
def train_and_evaluate(model, X_train, y_train, X_val, y_val, method_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    results[method_name] = {
        'accuracy': accuracy_score(y_val, y_pred),
        'report': classification_report(y_val, y_pred, output_dict=True)
    }
    print(f"--- {method_name} ---")
    print(f"Accuracy: {results[method_name]['accuracy']:.4f}")
    print(classification_report(y_val, y_pred))

# BoW Representation
print("Evaluating with BoW...")
X_train_bow = vectorizer.fit_transform(X_train)
# Transform the validation data using the same vectorizer fitted on training data
X_val_bow = vectorizer.transform(X_val)
# Train and evaluate using BoW
train_and_evaluate(LogisticRegression(max_iter=1000), X_train_bow, y_train, X_val_bow, y_val, "BoW")

# TF-IDF Representation
print("Evaluating with TF-IDF...")
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
# Transform the validation data using the same TF-IDF vectorizer fitted on training data
X_val_tfidf = tfidf_vectorizer.transform(X_val)
# Train and evaluate using TF-IDF
train_and_evaluate(LogisticRegression(max_iter=1000), X_train_tfidf, y_train, X_val_tfidf, y_val, "TF-IDF")

# Word2Vec Representation
print("Evaluating with Word2Vec...")
def get_avg_word2vec(tokens_list, model, vector_size):
    return np.array([np.mean([model[word] for word in tokens if word in model] or [np.zeros(vector_size)], axis=0)
                     for tokens in tokens_list])

X_train_tokens = merged_df.loc[X_train.index, 'tokens']
# Get tokens for validation data
X_val_tokens = merged_df.loc[X_val.index, 'tokens']
X_train_w2v = get_avg_word2vec(X_train_tokens, word_vectors, vector_size=100)
# Get Word2Vec embeddings for validation data
X_val_w2v = get_avg_word2vec(X_val_tokens, word_vectors, vector_size=100)
# Train and evaluate using Word2Vec
train_and_evaluate(LogisticRegression(max_iter=1000), X_train_w2v, y_train, X_val_w2v, y_val, "Word2Vec")

Evaluating with BoW...
--- BoW ---
Accuracy: 0.4367
              precision    recall  f1-score   support

       anger       0.55      0.06      0.11      7973
anticipation       0.52      0.37      0.43     49787
     disgust       0.32      0.15      0.21     27820
        fear       0.65      0.15      0.25     12800
         joy       0.43      0.84      0.57    103204
     sadness       0.36      0.25      0.30     38687
    surprise       0.42      0.06      0.10      9746
       trust       0.51      0.12      0.20     41096

    accuracy                           0.44    291113
   macro avg       0.47      0.25      0.27    291113
weighted avg       0.45      0.44      0.38    291113

Evaluating with TF-IDF...
--- TF-IDF ---
Accuracy: 0.4399
              precision    recall  f1-score   support

       anger       0.63      0.09      0.15      7973
anticipation       0.49      0.40      0.44     49787
     disgust       0.31      0.16      0.21     27820
        fear       0.6

### BERT

Since their performance are all below expectation, I also tried BERT + Transformers.

In [58]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import DistilBertForSequenceClassification
from transformers import EarlyStoppingCallback
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset
import wandb
wandb.init(mode="disabled")

# Encode the labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Custom Dataset
class EmotionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts.iloc[index]
        label = self.labels[index]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long),
        }

# Create datasets
train_dataset = EmotionDataset(X_train, y_train_encoded, tokenizer)

# Model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=8)

# Training arguments
training_args = TrainingArguments(
    gradient_accumulation_steps=4,
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    fp16=True,
    dataloader_num_workers=4,
)

# Create the test dataset
test_dataset = EmotionDataset(X_test, [0] * len(X_test), tokenizer)  # Use test labels as dummy labels for now

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-58-880b01a385f7>:75: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Preprocess the test set
test_dataset = EmotionDataset(X_test, [0]*len(X_test), tokenizer)  # Dummy labels

# Predict
predictions = trainer.predict(test_dataset)
predicted_classes = torch.argmax(torch.tensor(predictions.predictions), axis=1)

# Decode predictions
predicted_emotions = label_encoder.inverse_transform(predicted_classes.numpy())

# Save results
test_data["predicted_emotion"] = predicted_emotions
test_data[["tweet_id", "predicted_emotion"]].to_csv("predictions.csv", index=False)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
val_accuracy = accuracy_score(y_val, predicted_emotions)
print(f"Validation Accuracy: {val_accuracy:.4f}")

In [ ]:
submission = test_data[["tweet_id", "predicted_emotion"]]
submission.columns = ["id", "emotion"]
submission.to_csv("final_submission.csv", index=False)

In [ ]:
print(submission.head())

          id  emotion
2   0x28b412  disgust
4   0x2de201  disgust
9   0x218443  disgust
30  0x2939d5  disgust
33  0x26289a  disgust
